# Évaluation SmartRAG via traces Langfuse + Ragas
Ce notebook récupère les traces SmartRAG stockées dans Langfuse, les compare avec un jeu de données de référence (CSV),
et calcule les métriques Ragas pour évaluer différentes configurations SmartRAG.

**Prérequis** : 
- SmartRAG configuré et intégré avec Langfuse
- Traces SmartRAG disponibles dans Langfuse
- Fichier CSV avec questions/réponses de référence
- Python 3.10+, packages listés ci-dessous

## 0) Installation des dépendances

In [ ]:
# Installation des dépendances (dernières versions 2025)
# %pip install --upgrade pip
# %pip install langfuse>=3.0.0 ragas>=0.3.0 pandas matplotlib seaborn python-dotenv
# %pip install openai>=1.0.0 google-generativeai>=0.8.0 anthropic>=0.64.0

## 1) Configuration
Configurez vos variables d'environnement pour Langfuse et les chemins de fichiers.

**Variables principales :**
- `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_BASE_URL` : Connexion Langfuse
- `REFERENCE_CSV` : Chemin vers le fichier CSV avec questions/réponses de référence
- `OUTPUT_CSV` : Fichier de sortie avec les résultats d'évaluation
- `SMARTRAG_PROJECT_NAME` : Nom du projet SmartRAG dans Langfuse (optionnel)
- `EVALUATION_TIMERANGE` : Période d'évaluation en jours (défaut: 7 derniers jours)
- `RAGAS_LLM_PROVIDER` : Provider LLM pour Ragas (openai|gemini|claude|ollama)
- `RAGAS_MODEL_NAME` : Modèle à utiliser (gpt-4.1-mini, gemini-2.5-flash, claude-3-5-haiku-20241022)

In [3]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

load_dotenv()

# --- Configuration Langfuse ---
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY", "pk-lf-b65665b3-76ec-4a73-95ac-91254ae9af8a")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY", "your-openai-api-key")
LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")

# --- Configuration fichiers ---
REFERENCE_CSV = os.getenv("REFERENCE_CSV", "./data/reference/reference_qa.csv")
OUTPUT_CSV = os.getenv("OUTPUT_CSV", "./data/results/smartrag_evaluation_results.csv")

# --- Configuration SmartRAG ---
SMARTRAG_PROJECT_NAME = os.getenv("SMARTRAG_PROJECT_NAME", "")
EVALUATION_TIMERANGE = int(os.getenv("EVALUATION_TIMERANGE", "7"))  # jours

# --- Configuration évaluation ---
MIN_CONFIDENCE_SCORE = float(os.getenv("MIN_CONFIDENCE_SCORE", "0.7"))
INCLUDE_FAILED_TRACES = os.getenv("INCLUDE_FAILED_TRACES", "false").lower() == "true"

# --- Configuration Ragas LLM (2025 models) ---
RAGAS_LLM_PROVIDER = os.getenv("RAGAS_LLM_PROVIDER", "openai")
RAGAS_MODEL_NAME = os.getenv("RAGAS_MODEL_NAME", "gpt-4.1-mini")

print("Configuration chargée:")
print(f"LANGFUSE_BASE_URL: {LANGFUSE_BASE_URL}")
print(f"REFERENCE_CSV: {REFERENCE_CSV}")
print(f"OUTPUT_CSV: {OUTPUT_CSV}")
print(f"SMARTRAG_PROJECT_NAME: {SMARTRAG_PROJECT_NAME or 'Tous les projets'}")
print(f"EVALUATION_TIMERANGE: {EVALUATION_TIMERANGE} jours")
print(f"RAGAS_LLM_PROVIDER: {RAGAS_LLM_PROVIDER}")
print(f"RAGAS_MODEL_NAME: {RAGAS_MODEL_NAME}")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("LANGFUSE_PUBLIC_KEY et LANGFUSE_SECRET_KEY sont requis")

Configuration chargée:
LANGFUSE_BASE_URL: https://cloud.langfuse.com
REFERENCE_CSV: ./reference_qa_real.csv
OUTPUT_CSV: ./smartrag_evaluation_results.csv
SMARTRAG_PROJECT_NAME: Tous les projets
EVALUATION_TIMERANGE: 7 jours
RAGAS_LLM_PROVIDER: openai  # openai | gemini | claude | ollama
RAGAS_MODEL_NAME: gpt-4.1-mini  # ou gpt-4.1-mini, gemini-2.5-flash, claude-3-5-haiku-20241022, etc.


## 2) Structure du fichier CSV de référence

Le fichier CSV doit contenir les colonnes suivantes :
- `question_id` : Identifiant unique de la question
- `question` : Texte de la question
- `reference_answer` : Réponse de référence
- `expected_contexts` : Contextes attendus (optionnel, séparés par '|||')
- `category` : Catégorie de la question (optionnel)
- `difficulty` : Niveau de difficulté (optionnel)

Exemple de création d'un fichier de référence :

In [4]:
# Exemple : création d'un fichier CSV de référence (exécutez une seule fois)
sample_data = {
    'question_id': ['Q001', 'Q002', 'Q003'],
    'question': [
        'Quelle est la procédure de connexion au VPN ?',
        'Comment réinitialiser mon mot de passe ?',
        'Quels sont les horaires du support technique ?'
    ],
    'reference_answer': [
        'Pour vous connecter au VPN, utilisez le client Cisco AnyConnect avec vos identifiants AD.',
        'Pour réinitialiser votre mot de passe, contactez le service IT ou utilisez le portail self-service.',
        'Le support technique est disponible du lundi au vendredi de 8h à 18h.'
    ],
    'expected_contexts': [
        'Guide VPN|||Documentation réseau',
        'Procédure mot de passe|||Guide utilisateur',
        'Horaires support|||Contact IT'
    ],
    'category': ['Réseau', 'Sécurité', 'Support'],
    'difficulty': ['Facile', 'Moyen', 'Facile']
}

# Décommentez pour créer le fichier exemple
# sample_df = pd.DataFrame(sample_data)
# sample_df.to_csv('reference_qa_example.csv', index=False, encoding='utf-8')
# print("Fichier exemple créé : reference_qa_example.csv")

## 3) Chargement du jeu de référence

In [5]:
# Chargement du CSV de référence
try:
    df_reference = pd.read_csv(REFERENCE_CSV, encoding='utf-8')
    print(f"Jeu de référence chargé : {len(df_reference)} questions")
    
    # Validation des colonnes obligatoires
    required_cols = ['question_id', 'question', 'reference_answer']
    missing_cols = [col for col in required_cols if col not in df_reference.columns]
    if missing_cols:
        raise ValueError(f"Colonnes manquantes dans le CSV : {missing_cols}")
    
    # Nettoyage des données
    df_reference['question'] = df_reference['question'].fillna('').astype(str).str.strip()
    df_reference['reference_answer'] = df_reference['reference_answer'].fillna('').astype(str).str.strip()
    
    # Traitement des contextes attendus
    if 'expected_contexts' in df_reference.columns:
        df_reference['expected_contexts_list'] = df_reference['expected_contexts'].fillna('').apply(
            lambda x: x.split('|||') if x else []
        )
    else:
        df_reference['expected_contexts_list'] = [[] for _ in range(len(df_reference))]
    
    print("Aperçu du jeu de référence:")
    display(df_reference.head())
    
except FileNotFoundError:
    print(f"ERREUR : Fichier {REFERENCE_CSV} non trouvé.")
    print("Veuillez créer votre fichier CSV de référence ou ajuster la variable REFERENCE_CSV.")
    raise
except Exception as e:
    print(f"ERREUR lors du chargement du CSV : {e}")
    raise

ERREUR : Fichier ./reference_qa_real.csv non trouvé.
Veuillez créer votre fichier CSV de référence ou ajuster la variable REFERENCE_CSV.


FileNotFoundError: [Errno 2] No such file or directory: './reference_qa_real.csv'

## 4) Connexion à Langfuse et récupération des traces

In [ ]:
from langfuse import Langfuse

# Connexion à Langfuse
langfuse_client = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_BASE_URL
)

print("Connexion à Langfuse réussie")

# Définition de la période d'évaluation
end_date = datetime.now()
start_date = end_date - timedelta(days=EVALUATION_TIMERANGE)

print(f"Période d'évaluation : du {start_date.strftime('%Y-%m-%d')} au {end_date.strftime('%Y-%m-%d')}")

In [ ]:
def fetch_smartrag_traces(client: Langfuse, start_date: datetime, end_date: datetime, 
                        project_name: Optional[str] = None) -> List[Dict]:
    """Récupère les traces SmartRAG depuis Langfuse"""
    
    traces_data = []
    page = 1
    limit = 100
    
    print("Récupération des traces Langfuse...")
    
    while True:
        try:
            # Récupération des traces avec filtres
            traces = client.get_traces(
                page=page,
                limit=limit,
                from_timestamp=start_date,
                to_timestamp=end_date
            )
            
            if not traces.data:
                break
                
            print(f"Page {page} : {len(traces.data)} traces récupérées")
            
            for trace in traces.data:
                # Filtrage par nom de projet si spécifié
                if project_name and trace.name != project_name:
                    continue
                    
                # Extraction des informations de la trace
                trace_data = {
                    'trace_id': trace.id,
                    'timestamp': trace.timestamp,
                    'name': trace.name,
                    'input': trace.input,
                    'output': trace.output,
                    'metadata': trace.metadata or {},
                    'tags': trace.tags or [],
                    'user_id': trace.user_id,
                    'session_id': trace.session_id,
                    'version': trace.version,
                    'release': trace.release,
                    'public': trace.public
                }
                
                # Récupération des spans (étapes RAG)
                spans = client.get_trace(trace.id)
                if spans and hasattr(spans, 'observations'):
                    trace_data['observations'] = []
                    for obs in spans.observations:
                        obs_data = {
                            'id': obs.id,
                            'type': obs.type,
                            'name': obs.name,
                            'input': obs.input,
                            'output': obs.output,
                            'metadata': obs.metadata or {},
                            'start_time': obs.start_time,
                            'end_time': obs.end_time
                        }
                        trace_data['observations'].append(obs_data)
                
                traces_data.append(trace_data)
            
            page += 1
            
            # Protection contre les boucles infinies
            if page > 100:  # Max 10000 traces
                print("Limite de pages atteinte (100)")
                break
                
        except Exception as e:
            print(f"Erreur lors de la récupération des traces (page {page}): {e}")
            break
    
    print(f"Total des traces récupérées : {len(traces_data)}")
    return traces_data

# Récupération des traces
smartrag_traces = fetch_smartrag_traces(
    langfuse_client, 
    start_date, 
    end_date, 
    SMARTRAG_PROJECT_NAME if SMARTRAG_PROJECT_NAME else None
)

print(f"\nNombre de traces SmartRAG récupérées : {len(smartrag_traces)}")

## 5) Traitement et correspondance des traces

In [ ]:
def extract_rag_components(trace: Dict) -> Dict[str, Any]:
    """Extrait les composants RAG d'une trace Langfuse"""
    
    # Extraction de la question (input principal)
    question = ""
    if isinstance(trace.get('input'), str):
        question = trace['input']
    elif isinstance(trace.get('input'), dict):
        question = trace['input'].get('message', trace['input'].get('question', trace['input'].get('query', '')))
    
    # Extraction de la réponse (output principal)
    answer = ""
    if isinstance(trace.get('output'), str):
        answer = trace['output']
    elif isinstance(trace.get('output'), dict):
        answer = trace['output'].get('answer', trace['output'].get('response', trace['output'].get('text', '')))
    
    # Extraction des contextes depuis les observations
    contexts = []
    retrieval_info = {}
    generation_info = {}
    
    if 'observations' in trace:
        for obs in trace['observations']:
            obs_name = obs.get('name', '').lower()
            
            # Identification des étapes de retrieval
            if 'retriev' in obs_name or 'search' in obs_name or 'vector' in obs_name:
                if obs.get('output'):
                    if isinstance(obs['output'], list):
                        contexts.extend([str(item) for item in obs['output']])
                    elif isinstance(obs['output'], dict):
                        if 'documents' in obs['output']:
                            contexts.extend([str(doc) for doc in obs['output']['documents']])
                        elif 'results' in obs['output']:
                            contexts.extend([str(res) for res in obs['output']['results']])
                    else:
                        contexts.append(str(obs['output']))
                
                retrieval_info.update({
                    'retrieval_duration': obs.get('end_time', 0) - obs.get('start_time', 0) if obs.get('end_time') and obs.get('start_time') else None,
                    'retrieval_metadata': obs.get('metadata', {})
                })
            
            # Identification des étapes de génération
            elif 'generat' in obs_name or 'llm' in obs_name or 'model' in obs_name:
                generation_info.update({
                    'generation_duration': obs.get('end_time', 0) - obs.get('start_time', 0) if obs.get('end_time') and obs.get('start_time') else None,
                    'generation_metadata': obs.get('metadata', {}),
                    'model_name': obs.get('metadata', {}).get('model', 'unknown')
                })
    
    return {
        'trace_id': trace['trace_id'],
        'timestamp': trace['timestamp'],
        'question': question.strip(),
        'answer': answer.strip(),
        'contexts': contexts,
        'session_id': trace.get('session_id'),
        'user_id': trace.get('user_id'),
        'metadata': trace.get('metadata', {}),
        'tags': trace.get('tags', []),
        **retrieval_info,
        **generation_info
    }

# Traitement des traces
print("Traitement des traces SmartRAG...")
processed_traces = []

for trace in smartrag_traces:
    try:
        processed = extract_rag_components(trace)
        if processed['question'] and processed['answer']:  # On ne garde que les traces complètes
            processed_traces.append(processed)
    except Exception as e:
        print(f"Erreur lors du traitement de la trace {trace.get('trace_id', 'unknown')}: {e}")
        continue

print(f"Traces traitées avec succès : {len(processed_traces)}")

# Conversion en DataFrame
df_traces = pd.DataFrame(processed_traces)
if len(df_traces) > 0:
    print("\nAperçu des traces traitées:")
    display(df_traces[['trace_id', 'timestamp', 'question', 'answer']].head())
else:
    print("\nAucune trace valide trouvée.")
    print("Vérifiez :")
    print("- Que SmartRAG envoie bien les traces à Langfuse")
    print("- La période d'évaluation (EVALUATION_TIMERANGE)")
    print("- Le nom du projet SmartRAG (SMARTRAG_PROJECT_NAME)")

## 6) Correspondance questions de référence ↔ traces SmartRAG

In [ ]:
from difflib import SequenceMatcher

def calculate_similarity(text1: str, text2: str) -> float:
    """Calcule la similarité entre deux textes"""
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

def match_traces_to_reference(df_traces: pd.DataFrame, df_reference: pd.DataFrame, 
                            similarity_threshold: float = 0.8) -> pd.DataFrame:
    """Fait correspondre les traces aux questions de référence"""
    
    matched_data = []
    
    for _, ref_row in df_reference.iterrows():
        ref_question = ref_row['question']
        best_match = None
        best_similarity = 0
        
        # Recherche de la meilleure correspondance
        for _, trace_row in df_traces.iterrows():
            similarity = calculate_similarity(ref_question, trace_row['question'])
            
            if similarity > best_similarity and similarity >= similarity_threshold:
                best_similarity = similarity
                best_match = trace_row
        
        if best_match is not None:
            matched_row = {
                # Données de référence
                'question_id': ref_row['question_id'],
                'reference_question': ref_row['question'],
                'reference_answer': ref_row['reference_answer'],
                'expected_contexts': ref_row.get('expected_contexts_list', []),
                'category': ref_row.get('category', ''),
                'difficulty': ref_row.get('difficulty', ''),
                
                # Données de la trace
                'trace_id': best_match['trace_id'],
                'actual_question': best_match['question'],
                'actual_answer': best_match['answer'],
                'retrieved_contexts': best_match['contexts'],
                'question_similarity': best_similarity,
                
                # Métadonnées de la trace
                'timestamp': best_match['timestamp'],
                'session_id': best_match.get('session_id'),
                'user_id': best_match.get('user_id'),
                'retrieval_duration': best_match.get('retrieval_duration'),
                'generation_duration': best_match.get('generation_duration'),
                'model_name': best_match.get('model_name', 'unknown'),
                'trace_metadata': best_match.get('metadata', {}),
                'trace_tags': best_match.get('tags', [])
            }
            matched_data.append(matched_row)
        else:
            print(f"Pas de correspondance trouvée pour la question: {ref_row['question_id']} (seuil: {similarity_threshold})")
    
    return pd.DataFrame(matched_data)

# Correspondance des traces
if len(df_traces) > 0:
    print("Correspondance des traces aux questions de référence...")
    df_matched = match_traces_to_reference(df_traces, df_reference, similarity_threshold=0.7)
    
    print(f"\nCorrespondances trouvées : {len(df_matched)}/{len(df_reference)}")
    
    if len(df_matched) > 0:
        print(f"Similarité moyenne des questions : {df_matched['question_similarity'].mean():.3f}")
        display(df_matched[['question_id', 'question_similarity', 'reference_question', 'actual_question']].head())
    else:
        print("Aucune correspondance trouvée. Essayez de réduire le seuil de similarité ou vérifiez les questions.")
else:
    print("Impossible de faire les correspondances : aucune trace disponible.")
    df_matched = pd.DataFrame()

## 7) Évaluation Ragas

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entities_recall, noise_sensitivity

if len(df_matched) > 0:
    print(f"Évaluation Ragas de {len(df_matched)} questions...")
    
    # Préparation des données pour Ragas
    eval_data = df_matched.copy()
    
    # Vérification et nettoyage des contextes
    eval_data['contexts_cleaned'] = eval_data['retrieved_contexts'].apply(
        lambda x: [str(ctx).strip() for ctx in x] if isinstance(x, list) and x else ["Aucun contexte"]
    )
    
    # Mapping des colonnes pour Ragas
    column_map = {
        "question": "reference_question",
        "answer": "actual_answer",
        "contexts": "contexts_cleaned",
        "ground_truth": "reference_answer"
    }
    
    # Sélection des métriques
    metrics = [
        faithfulness,           # Fidélité au contexte
        answer_relevancy,       # Pertinence de la réponse
        context_precision,      # Précision du contexte
        context_recall,         # Rappel du contexte
        context_entities_recall, # Rappel des entités
        noise_sensitivity       # Sensibilité au bruit
    ]
    
    try:
        # Évaluation Ragas
        ragas_results = evaluate(eval_data, metrics=metrics, column_map=column_map)
        
        print("\n=== RÉSULTATS RAGAS ===")
        print("\nScores moyens:")
        for metric, score in ragas_results.scores.items():
            print(f"  {metric}: {score:.4f}")
        
        # Fusion des résultats avec les données matchées
        df_final = df_matched.merge(ragas_results.results, left_index=True, right_index=True, how='left')
        
        print(f"\nÉvaluation terminée pour {len(df_final)} questions")
        
    except Exception as e:
        print(f"Erreur lors de l'évaluation Ragas: {e}")
        df_final = df_matched.copy()
        # Ajout de colonnes vides pour les métriques
        for metric in ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 
                      'context_entities_recall', 'noise_sensitivity']:
            df_final[metric] = None
else:
    print("Aucune donnée à évaluer.")
    df_final = pd.DataFrame()

## 8) Analyse et visualisations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

if len(df_final) > 0 and 'faithfulness' in df_final.columns:
    plt.figure(figsize=(15, 10))
    
    # Graphique 1 : Scores moyens par métrique
    plt.subplot(2, 3, 1)
    metrics_cols = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 
                   'context_entities_recall', 'noise_sensitivity']
    
    available_metrics = [col for col in metrics_cols if col in df_final.columns and df_final[col].notna().any()]
    
    if available_metrics:
        means = [df_final[col].mean() for col in available_metrics]
        plt.bar(range(len(available_metrics)), means, color='skyblue')
        plt.xticks(range(len(available_metrics)), available_metrics, rotation=45, ha='right')
        plt.title('Scores Ragas moyens')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        
        for i, v in enumerate(means):
            plt.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')
    
    # Graphique 2 : Distribution des scores de fidélité
    plt.subplot(2, 3, 2)
    if 'faithfulness' in df_final.columns and df_final['faithfulness'].notna().any():
        plt.hist(df_final['faithfulness'].dropna(), bins=20, color='lightcoral', alpha=0.7)
        plt.title('Distribution - Faithfulness')
        plt.xlabel('Score')
        plt.ylabel('Fréquence')
    
    # Graphique 3 : Scores par catégorie (si disponible)
    plt.subplot(2, 3, 3)
    if 'category' in df_final.columns and df_final['category'].notna().any() and 'faithfulness' in df_final.columns:
        category_scores = df_final.groupby('category')['faithfulness'].mean().sort_values(ascending=False)
        plt.bar(range(len(category_scores)), category_scores.values, color='lightgreen')
        plt.xticks(range(len(category_scores)), category_scores.index, rotation=45, ha='right')
        plt.title('Faithfulness par catégorie')
        plt.ylabel('Score moyen')
    
    # Graphique 4 : Corrélation similarité de question vs. qualité de réponse
    plt.subplot(2, 3, 4)
    if 'question_similarity' in df_final.columns and 'answer_relevancy' in df_final.columns:
        valid_data = df_final[df_final['answer_relevancy'].notna() & df_final['question_similarity'].notna()]
        if len(valid_data) > 0:
            plt.scatter(valid_data['question_similarity'], valid_data['answer_relevancy'], alpha=0.6)
            plt.xlabel('Similarité question')
            plt.ylabel('Answer Relevancy')
            plt.title('Similarité vs. Pertinence')
    
    # Graphique 5 : Performance par modèle
    plt.subplot(2, 3, 5)
    if 'model_name' in df_final.columns and 'faithfulness' in df_final.columns:
        model_scores = df_final.groupby('model_name')['faithfulness'].mean().sort_values(ascending=False)
        if len(model_scores) > 1:
            plt.bar(range(len(model_scores)), model_scores.values, color='gold')
            plt.xticks(range(len(model_scores)), model_scores.index, rotation=45, ha='right')
            plt.title('Faithfulness par modèle')
            plt.ylabel('Score moyen')
    
    # Graphique 6 : Évolution temporelle (si plusieurs jours)
    plt.subplot(2, 3, 6)
    if 'timestamp' in df_final.columns and len(df_final) > 1:
        df_final['date'] = pd.to_datetime(df_final['timestamp']).dt.date
        daily_scores = df_final.groupby('date')['faithfulness'].mean()
        if len(daily_scores) > 1:
            plt.plot(daily_scores.index, daily_scores.values, marker='o', color='purple')
            plt.title('Évolution Faithfulness')
            plt.xlabel('Date')
            plt.ylabel('Score moyen')
            plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques détaillées
    print("\n=== STATISTIQUES DÉTAILLÉES ===")
    
    for metric in available_metrics:
        values = df_final[metric].dropna()
        if len(values) > 0:
            print(f"\n{metric.upper()}:")
            print(f"  Moyenne: {values.mean():.4f}")
            print(f"  Médiane: {values.median():.4f}")
            print(f"  Écart-type: {values.std():.4f}")
            print(f"  Min: {values.min():.4f}")
            print(f"  Max: {values.max():.4f}")
            
            # Questions avec les plus mauvais scores
            worst_idx = values.idxmin()
            print(f"  Pire score: Question {df_final.loc[worst_idx, 'question_id']} ({values.min():.4f})")
            
            # Questions avec les meilleurs scores
            best_idx = values.idxmax()
            print(f"  Meilleur score: Question {df_final.loc[best_idx, 'question_id']} ({values.max():.4f})")

else:
    print("Aucune donnée disponible pour les visualisations.")

## 9) Export des résultats

In [ ]:
if len(df_final) > 0:
    # Préparation des colonnes d'export
    export_columns = [
        'question_id', 'category', 'difficulty', 'question_similarity',
        'reference_question', 'actual_question', 'reference_answer', 'actual_answer',
        'faithfulness', 'answer_relevancy', 'context_precision', 'context_recall',
        'context_entities_recall', 'noise_sensitivity',
        'trace_id', 'timestamp', 'session_id', 'user_id', 'model_name',
        'retrieval_duration', 'generation_duration'
    ]
    
    # Filtrage des colonnes existantes
    available_columns = [col for col in export_columns if col in df_final.columns]
    
    # Export CSV
    df_export = df_final[available_columns].copy()
    df_export.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
    
    print(f"Résultats exportés vers : {OUTPUT_CSV}")
    print(f"Colonnes exportées : {len(available_columns)}")
    print(f"Lignes exportées : {len(df_export)}")
    
    # Export JSON pour analyse approfondie
    json_output = OUTPUT_CSV.replace('.csv', '_detailed.json')
    
    # Préparation des données JSON avec plus de détails
    json_data = {
        'evaluation_metadata': {
            'evaluation_date': datetime.now().isoformat(),
            'evaluation_period': {
                'start': start_date.isoformat(),
                'end': end_date.isoformat()
            },
            'total_reference_questions': len(df_reference),
            'matched_questions': len(df_final),
            'smartrag_project': SMARTRAG_PROJECT_NAME or 'All projects',
            'langfuse_url': LANGFUSE_BASE_URL
        },
        'summary_metrics': {},
        'detailed_results': df_final.to_dict('records')
    }
    
    # Calcul des métriques de résumé
    ragas_metrics = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 
                    'context_entities_recall', 'noise_sensitivity']
    
    for metric in ragas_metrics:
        if metric in df_final.columns and df_final[metric].notna().any():
            values = df_final[metric].dropna()
            json_data['summary_metrics'][metric] = {
                'mean': float(values.mean()),
                'median': float(values.median()),
                'std': float(values.std()),
                'min': float(values.min()),
                'max': float(values.max()),
                'count': int(len(values))
            }
    
    # Sauvegarde JSON
    with open(json_output, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    print(f"Analyse détaillée exportée vers : {json_output}")
    
    # Aperçu des résultats
    print("\n=== APERÇU DES RÉSULTATS EXPORTÉS ===")
    display(df_export.head())

else:
    print("Aucune donnée à exporter.")
    print("\nPoints de vérification :")
    print("1. Vérifiez que SmartRAG envoie bien les traces à Langfuse")
    print("2. Ajustez la période d'évaluation (EVALUATION_TIMERANGE)")
    print("3. Vérifiez le nom du projet SmartRAG (SMARTRAG_PROJECT_NAME)")
    print("4. Vérifiez que les questions dans votre CSV correspondent aux questions posées à SmartRAG")

## 10) Conclusions et prochaines étapes

Ce notebook vous permet de :
1. **Récupérer** automatiquement les traces SmartRAG depuis Langfuse
2. **Comparer** les réponses générées avec vos données de référence
3. **Évaluer** la qualité avec les métriques Ragas
4. **Analyser** les performances par catégorie, modèle, période
5. **Exporter** les résultats pour analyse approfondie

### Prochaines étapes suggérées :

- **Itération sur les configurations SmartRAG** : Modifiez les paramètres de SmartRAG (chunking, embedding, reranking) et relancez l'évaluation
- **Comparaison A/B** : Utilisez les sessions Langfuse pour comparer différentes versions de votre pipeline
- **Monitoring continu** : Automatisez ce notebook pour un monitoring régulier de la qualité
- **Analyse des échecs** : Identifiez les questions avec les plus mauvais scores pour améliorer le système
- **Extension du jeu de test** : Enrichissez votre CSV de référence avec plus de questions représentatives

### Optimisation des performances :
- Ajustez le seuil de similarité pour la correspondance des questions
- Utilisez les tags Langfuse pour filtrer par version ou configuration
- Exploitez les métadonnées pour analyser les performances par type de document ou utilisateur